# Fashion compatibility - Data Exploration and Analysis

---
## Author Information
- **Author:** Francesco Tedesco
- **Email:** francescotedesco7d2@gmail.com
- [**LinkedIn**](https://www.linkedin.com/in/francescotedesco7d2/)

---
## Overview
This notebook explores the dataset to gain insights into the different types of outfits, in order to build a set of possible combinations of outfits for outfits.

---


In [1]:
import pandas as pd

from pathlib import Path
import os

os.chdir(str(Path.cwd().parent))

from src.utils.setup_utilities import load_config
from src.data_processing.group_manipulation import get_grouped_counts
from src.data_processing.group_manipulation import get_grouped_counts_feature_values
from src.data_processing.group_manipulation import get_unique_sets_features
from src.data_processing.group_manipulation import create_combinations
from src.data_processing.group_manipulation import create_configurations
from src.data_processing.group_manipulation import select_valid_outfits
from src.data_processing.product_processing import get_des_product_class

config = load_config()

In [2]:
df_outfits = pd.read_csv(config['data']['outfits_path'])
df_products = pd.read_csv(config['data']['products_path'])
df_outfit_products = pd.merge(df_outfits, df_products, on = 'cod_modelo_color', how = 'outer')

## Initial exploration

In [3]:
df_outfits.head()

,cod_outfit,cod_modelo_color
0,1,51000622-02
1,1,43067759-01
2,1,53060518-02
3,1,53030594-08
4,1,43077762-01


The `df_outfits` dataframe contains the codes of each outfit and the products code associated. 

In [4]:
df_products.head()

,cod_modelo_color,cod_color_code,des_color_specification_esp,des_agrup_color_eng,des_sex,des_age,des_line,des_fabric,des_product_category,des_product_aggregated_family,des_product_family,des_product_type,des_filename
0,41085800-02,02,OFFWHITE,WHITE,Female,Adult,SHE,P-PLANA,Bottoms,Trousers & leggings,Trousers,Trousers,datathon/images/2019_41085800_02.jpg
1,53000586-TO,TO,TEJANO OSCURO,BLUE,Female,Adult,SHE,J-JEANS,Bottoms,Jeans,Jeans,Jeans,datathon/images/2019_53000586_TO.jpg
2,53030601-81,81,ROSA PASTEL,PINK,Female,Adult,SHE,P-PLANA,"Dresses, jumpsuits and Complete set",Dresses and jumpsuits,Dresses,Dress,datathon/images/2019_53030601_81.jpg
3,53050730-15,15,MOSTAZA,YELLOW,Female,Adult,SHE,P-PLANA,"Dresses, jumpsuits and Complete set",Dresses and jumpsuits,Dresses,Dress,datathon/images/2019_53050730_15.jpg
4,53070773-70,70,ROJO,RED,Female,Adult,SHE,P-PLANA,Tops,Shirts,Shirt,Shirt,datathon/images/2019_53070773_70.jpg


The `df_products` contains information about each product, including the path to the corresponding images.

Some important things to consider: 

In [5]:
print("Total number of outfits:", len(df_outfits['cod_outfit'].unique()))

Total number of outfits: 7842


In [6]:
print("Total number of products:", len(df_products['cod_modelo_color'].unique()))

Total number of products: 9222


In [7]:
existing_products = set(df_products['cod_modelo_color'].unique())
products_w_outfit = set(df_outfits['cod_modelo_color'].unique())

products_without_outfit = existing_products - products_w_outfit

print("Number of products without an outfit:", len(products_without_outfit))

Number of products without an outfit: 0


In [8]:
name_set = 'outfit'
code_sets = 'cod_outfit'
get_grouped_counts(df_outfits, code_sets, name_set)

,outfit_size,grouped_outfit_size
0,2,30
1,3,142
2,4,758
3,5,3875
4,6,1737
5,7,674
6,8,309
7,9,174
8,10,95
9,11,26


If we plot the number of outfits for each size, we can observe that there are potentially some outfits that can be excluded from the training set due to their low frequency.

In [9]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9222 entries, 0 to 9221
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   cod_modelo_color               9222 non-null   object
 1   cod_color_code                 9222 non-null   object
 2   des_color_specification_esp    9222 non-null   object
 3   des_agrup_color_eng            9222 non-null   object
 4   des_sex                        9222 non-null   object
 5   des_age                        9222 non-null   object
 6   des_line                       9222 non-null   object
 7   des_fabric                     9222 non-null   object
 8   des_product_category           9222 non-null   object
 9   des_product_aggregated_family  9222 non-null   object
 10  des_product_family             9222 non-null   object
 11  des_product_type               9222 non-null   object
 12  des_filename                   9222 non-null   object
dtypes: 

## Product description

Since not all outfits will be considered in the training process, we should start by examining their descriptions to determine which descriptors are useful. Later, we can use this information to establish criteria for defining valid combinations. 

In [10]:
df_products['des_product_category'].unique()

array(['Bottoms', 'Dresses, jumpsuits and Complete set', 'Tops',
       'Accesories, Swim and Intimate', 'Outerwear', 'Beauty', 'Home'],
      dtype=object)

In [11]:
df_products['des_product_family'].unique()

array(['Trousers', 'Jeans', 'Dresses', 'Shirt', 'Sweater', 'Skirts',
       'Jewellery', 'Bags', 'Glasses', 'Wallets & cases', 'Shorts',
       'Tops', 'Belts and Ties', 'Jumpsuit', 'Jackets', 'Coats',
       'Footwear', 'Hats, scarves and gloves', 'T-shirt', 'Blazers',
       'Gadgets', 'Swimwear', 'Vest', 'Fragances', 'Cardigans',
       'Trenchcoats', 'Puffer coats', 'Outer Vest',
       'Leggings and joggers', 'Deco Accessories', 'Poloshirts',
       'Intimate', 'Sweatshirts', 'Deco Textiles', 'Bedding', 'Bodysuits',
       'Leather jackets', 'Parkas', 'Glassware'], dtype=object)

In [12]:
df_products['des_product_type'].unique()

array(['Trousers', 'Jeans', 'Dress', 'Shirt', 'Sweater', 'Skirt',
       'Earrings', 'Totes bag', 'Sunglasses', 'Card holder', 'Wallet',
       'Shorts', 'Top', 'Belt', 'Crossbody bag', 'Jumpsuit', 'Jacket',
       'Coat', 'Sandals', 'Kerchief', 'Shoes', 'Blouse', 'T-Shirt',
       'Blazer', 'Umbrella', 'Citybag', 'Bikini top', 'Vest',
       'Shoulder bag', 'Bodymist', 'Beanie', 'Handbag', 'Cardigan',
       'Glasses', 'Trenchcoat', 'Puffer coat', 'Necklace',
       'Bikini pantie', 'Outer vest', 'Scarf', 'Ankle Boots', 'Leggings',
       'Basket', 'Cosmetic bag', 'Ring', 'Poloshirt', 'Pyjama',
       'Sweatshirt', 'Plaid', 'Boots', 'Hat', 'Duvet Covers',
       'Beach Towel', 'Gloves', 'Bodysuit', 'Fragance', 'Leather Jacket',
       'Hairband', 'Bermudas', 'Cap', 'Parka', 'Pyjama Trousers',
       'Pyjama Shirt', 'Bras', 'Trainers', 'Foulard', 'Hairclip', 'Case',
       'Bracelet', 'Pyjama Shorts', 'Sweater Vest', 'Pyjiama Sweater',
       'Bucket bag', 'Jacket (Cazadora)', 'Purse',

We can observe that as the description becomes more specific, the number of different labels increases. Let's examine the counts for the 'des_product_category' (less specific description) values:

In [13]:
df_outfit_products['des_product_category'].value_counts()

des_product_category
Accesories, Swim and Intimate          27315
Tops                                    6313
Bottoms                                 5839
Outerwear                               2080
Dresses, jumpsuits and Complete set     1777
Home                                     169
Beauty                                    89
Name: count, dtype: int64

As Home and Beauty labels are less common, we can exclude them when defining valid outfit combinations. The other labels are more generalizable, and they are less likely to contain 'strange' products. This cannot be said for the most common label (Accesories, Swim and Intimate), where sublabels may potentially have very low frequency and/or make less sense for products.

In [14]:
df_accesories = df_outfit_products[df_outfit_products['des_product_category'] == 'Accesories, Swim and Intimate']
df_accesories['des_product_family'].value_counts()

des_product_family
Jewellery                   9686
Footwear                    7976
Bags                        6152
Glasses                      910
Belts and Ties               767
Wallets & cases              611
Hats, scarves and gloves     551
Intimate                     473
Swimwear                     158
Gadgets                       31
Name: count, dtype: int64

The same observation can be made here for the jewelry family. Let's examine the corresponding subtypes.

In [15]:
df_accesories = df_outfit_products[df_outfit_products['des_product_category'] == 'Accesories, Swim and Intimate']
df_jewellery = df_accesories[df_accesories['des_product_family'] == 'Jewellery']
df_jewellery['des_product_type'].value_counts()

des_product_type
Earrings    5858
Ring        1832
Necklace    1363
Bracelet     614
Hairclip      19
Name: count, dtype: int64

We can now create a personalized product class by combining the three column descriptors we just examined.

In [16]:
df_outfit_products = df_outfit_products.copy()
df_outfit_products['des_product_class'] = df_outfit_products.apply(get_des_product_class, axis=1)

In [17]:
df_outfit_products['des_product_class'].unique()

array(['Footwear', 'Tops', 'Bags', 'Earrings', 'Bottoms', 'Outerwear',
       'Dresses, jumpsuits and Complete set', 'Necklace',
       'Hats, scarves and gloves', 'Belts and Ties', 'Bracelet',
       'Swimwear', 'Glasses', 'Ring', 'Gadgets', 'Wallets & cases',
       'Home', 'Intimate', 'Beauty', 'Hairclip'], dtype=object)

## Outfits structure 


Now that we've created a new class to describe the products, let's take a closer look at how the outfits are actually constructed. 


In [18]:
size = 2
feature_name = 'des_product_class'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_class_set,des_product_class_outfit_codes,des_product_class_count
1,{'Wallets & cases': 2},"[1571, 1572, 1573, 1574, 1580, 1620, 1621, 203...",14
3,{'Intimate': 2},"[2711, 2727, 5794, 5795]",4
4,"{'Tops': 1, 'Bottoms': 1}","[3712, 3755, 3818, 7373]",4
0,"{'Tops': 1, 'Earrings': 1}","[1214, 3872]",2
5,"{'Footwear': 1, 'Intimate': 1}","[4034, 4496]",2


In [19]:
size = 3
feature_name = 'des_product_class'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_class_set,des_product_class_outfit_codes,des_product_class_count
11,"{'Dresses, jumpsuits and Complete set': 1, 'Ea...","[942, 1333, 1504, 1582, 1594, 1809, 2068, 2094...",18
4,{'Wallets & cases': 3},"[591, 810, 813, 990, 1384, 1557, 1684, 1759, 1...",13
24,"{'Hats, scarves and gloves': 3}","[1318, 1319, 1320, 1481, 1482, 1483, 2091, 209...",9
15,"{'Dresses, jumpsuits and Complete set': 1, 'Ba...","[1019, 1593, 3368, 5464, 5479, 6421, 6949, 696...",9
17,"{'Glasses': 1, 'Bags': 1, 'Footwear': 1}","[1141, 1536, 1567, 1709, 1710, 2102, 2335, 6417]",8


In [20]:
size = 4
feature_name = 'des_product_class'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_class_set,des_product_class_outfit_codes,des_product_class_count
3,"{'Dresses, jumpsuits and Complete set': 1, 'Ea...","[55, 160, 191, 229, 231, 232, 249, 251, 252, 2...",210
2,"{'Bottoms': 1, 'Tops': 1, 'Earrings': 1, 'Foot...","[39, 185, 288, 350, 352, 369, 425, 428, 463, 4...",90
16,"{'Glasses': 1, 'Bags': 1, 'Footwear': 1, 'Ring...","[609, 709, 895, 982, 987, 988, 1037, 1048, 107...",42
9,"{'Tops': 1, 'Bottoms': 1, 'Bags': 1, 'Footwear...","[312, 366, 570, 574, 688, 701, 789, 804, 917, ...",34
26,"{'Earrings': 1, 'Belts and Ties': 1, 'Footwear...","[883, 888, 964, 1008, 1009, 1092, 1385, 1597, ...",21


In [21]:
size = 5
feature_name = 'des_product_type'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_type_set,des_product_type_outfit_codes,des_product_type_count
3,"{'Dress': 1, 'Handbag': 1, 'Sandals': 1, 'Earr...","[15, 22, 23, 24, 33, 37, 153, 163, 228, 353, 3...",54
444,"{'Crossbody bag': 1, 'Dress': 1, 'Ring': 1, 'S...","[2600, 2601, 2804, 2914, 3336, 3538, 4052, 426...",35
7,"{'Necklace': 1, 'Dress': 1, 'Handbag': 1, 'Ear...","[32, 57, 324, 516, 645, 825, 862, 1016, 1231, ...",35
440,"{'Shoulder bag': 1, 'Earrings': 1, 'Dress': 1,...","[2592, 2631, 2821, 3405, 3448, 3480, 3685, 401...",32
459,"{'Sunglasses': 1, 'Sandals': 1, 'Earrings': 1,...","[2627, 2644, 2699, 2776, 2778, 2817, 2823, 346...",31


In [22]:
size = 6
feature_name = 'des_product_category'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_category_set,des_product_category_outfit_codes,des_product_category_count
1,"{'Accesories, Swim and Intimate': 4, 'Tops': 1...","[7, 27, 28, 30, 44, 45, 51, 58, 60, 62, 70, 82...",587
3,"{'Bottoms': 1, 'Outerwear': 1, 'Accesories, Sw...","[40, 67, 69, 76, 79, 80, 81, 83, 84, 105, 114,...",558
2,"{'Dresses, jumpsuits and Complete set': 1, 'Ac...","[13, 77, 98, 200, 212, 213, 221, 239, 267, 274...",181
4,"{'Bottoms': 1, 'Tops': 2, 'Accesories, Swim an...","[50, 65, 137, 408, 417, 495, 712, 770, 790, 85...",169
13,"{'Bottoms': 2, 'Tops': 1, 'Accesories, Swim an...","[1029, 1321, 2183, 2320, 2580, 2814, 3022, 304...",39


In [23]:
size = 7
feature_name = 'des_product_class'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_class_set,des_product_class_outfit_codes,des_product_class_count
11,"{'Bottoms': 1, 'Tops': 1, 'Earrings': 1, 'Foot...","[72, 99, 202, 219, 244, 333, 382, 389, 572, 65...",41
5,"{'Earrings': 1, 'Outerwear': 1, 'Bottoms': 1, ...","[35, 48, 68, 87, 111, 188, 237, 276, 342, 413,...",41
191,"{'Home': 5, 'Intimate': 2}","[4065, 4702, 4703, 5245, 5246, 5355, 5356, 543...",20
116,"{'Bags': 1, 'Footwear': 1, 'Outerwear': 1, 'Bo...","[2072, 2539, 3147, 3201, 3264, 3301, 4275, 504...",18
43,"{'Bottoms': 1, 'Tops': 1, 'Ring': 2, 'Bags': 1...","[410, 518, 1035, 1051, 1127, 1442, 1705, 1706,...",18


In [24]:
size = 8
feature_name = 'des_product_class'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_class_set,des_product_class_outfit_codes,des_product_class_count
10,"{'Bottoms': 1, 'Outerwear': 1, 'Footwear': 1, ...","[172, 455, 898, 963, 1360, 1477, 1495, 1944, 2...",11
48,"{'Bottoms': 2, 'Footwear': 2, 'Tops': 1, 'Earr...","[838, 3421, 5206, 5888, 5969, 6370, 6670, 7581...",9
108,{'Wallets & cases': 8},"[2638, 3415, 3994, 4006, 4445, 4460, 4952, 5656]",8
111,"{'Bags': 1, 'Tops': 2, 'Earrings': 1, 'Intimat...","[2915, 3241, 4118, 5275, 6023, 6024, 6847, 6853]",8
106,"{'Tops': 2, 'Footwear': 2, 'Bags': 1, 'Outerwe...","[2504, 5500, 5603, 6248, 6637, 6758, 7549]",7


In [25]:
size = 9
feature_name = 'des_product_class'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_class_set,des_product_class_outfit_codes,des_product_class_count
36,"{'Earrings': 4, 'Ring': 3, 'Necklace': 2}","[1045, 1046, 1047, 1264, 1417, 1849, 2050, 226...",9
72,"{'Bags': 1, 'Footwear': 2, 'Tops': 3, 'Bottoms...","[3038, 3102, 3211]",3
86,"{'Footwear': 2, 'Tops': 3, 'Bags': 1, 'Earring...","[4101, 4102, 6270]",3
85,"{'Bags': 2, 'Tops': 2, 'Outerwear': 1, 'Footwe...","[4074, 5230, 5231]",3
1,"{'Bottoms': 1, 'Outerwear': 1, 'Tops': 1, 'Ear...","[47, 421, 765]",3


In [26]:
size = 10
feature_name = 'des_product_class'
name_set = 'outfit'
code_sets = 'cod_outfit'

result_df = get_grouped_counts_feature_values(df_outfit_products, code_sets, name_set, feature_name)
tuple_set = result_df[result_df['outfit_size'] == size][f'cod_outfit_{feature_name}_tuple'].iloc[0]
features_sets = get_unique_sets_features(tuple_set, feature_name)
features_sets.head()

,des_product_class_set,des_product_class_outfit_codes,des_product_class_count
44,"{'Bags': 2, 'Wallets & cases': 8}","[2606, 2637, 4002, 4454, 4456, 4960, 5661, 5662]",8
2,"{'Bottoms': 1, 'Earrings': 1, 'Necklace': 2, '...","[92, 204, 607, 628]",4
6,"{'Earrings': 1, 'Bags': 2, 'Bracelet': 1, 'Top...","[297, 348]",2
58,"{'Bags': 1, 'Tops': 3, 'Bottoms': 1, 'Belts an...","[4083, 4084]",2
19,"{'Tops': 2, 'Bottoms': 1, 'Earrings': 2, 'Neck...","[1024, 1235]",2


# Defining valid outfit combinations

Upon reviewing all the outfits, we can establish some rules for selecting valid combinations:

In addition to these observations, we can define a rule for constructing the outfits:

    The foundation of the outfits must consist of one of the following sets:
        Tops + Bottoms + Footwear + Earrings + Accessories
        Dress + Footwear + Earrings + Accessories

    The outfit can optionally include the following complements:
        Outerwear, Bags, Glasses, Ring, and Necklace

We get, that a valid outfit is a configuration base plus (or
not) a possible configuration of complements, each comple-
ment, cannot appear more than once in the configuration.
And so we get 42 possible configuration for the outfits, and
each outfit can reach up to 10 products of different classes


NOTE: Looking at outfits with sizes 2 or higher or equal than
9, we observe that didn’t reach the conditions for being
a valid outfit.

In [27]:
configurations_base = config['data']['configurations_base']

optional_products = config['data']['optional_products']

optional_configurations = create_combinations(optional_products, root = False)

all_configurations = create_configurations(configurations_base, optional_products)

print("Number of Total possible configurations of BASE products:", len(configurations_base))
print("Number of Total possible configurations of OPTIONAL products:", len(optional_configurations))
print("Number of Total possible configurations:", len(all_configurations))

Number of Total possible configurations of BASE products: 2
Number of Total possible configurations of OPTIONAL products: 20
Number of Total possible configurations: 42


In [28]:
feature_name = 'des_product_class'
code_sets = 'cod_outfit'
name_set = 'outfit'

df_outfit_products_sel, configurations_count = select_valid_outfits(df_outfit_products, feature_name, code_sets, name_set, all_configurations)

print("Total number of outfits selected:", len(df_outfit_products_sel['cod_outfit'].unique()))

df_outfit_products_sel.head()


Total number of outfits selected: 2303


,cod_outfit,cod_modelo_color,des_product_class,des_filename
7,2811,37000577-30,Footwear,datathon/images/2022_37000577_30.jpg
12,5685,37000577-30,Footwear,datathon/images/2022_37000577_30.jpg
15,5112,37010681-99,Footwear,datathon/images/2022_37010681_99.jpg
18,3475,37010681-99,Footwear,datathon/images/2022_37010681_99.jpg
20,2732,37010684-99,Footwear,datathon/images/2022_37010684_99.jpg


After selecting the valid outfits, we observe that only a minority of them can be considered trainable. Therefore, the model's performance will be evaluated based on its ability to distinguish good outfits from randomly created outfits using unseen products (unseen images). 


When randomly creating new outfits for evaluating the model later, the products will be selected at random based on the types of configurations considered during training:

In [29]:
configurations_count_df = pd.DataFrame(list(configurations_count.items()), columns=['conf_id', 'size_conf'])
configurations_count_df['conf_id'] = configurations_count_df['conf_id'].astype(int)
configurations_count_df['prob_conf'] = configurations_count_df['size_conf'] / sum(configurations_count_df['size_conf'])
configurations_count_df

,conf_id,size_conf,prob_conf
0,0,18,0.007816
1,21,90,0.039079
2,17,210,0.091185
3,19,16,0.006947
4,1,8,0.003474
5,18,5,0.002171
6,20,2,0.000868
7,38,1186,0.514980
8,40,64,0.027790
9,22,181,0.078593


Some additional information about the outfits and products considered in the training set is as follows:

In [30]:
name_set = 'outfit'
code_sets = 'cod_outfit'
get_grouped_counts(df_outfit_products_sel, code_sets, name_set)

,outfit_size,grouped_outfit_size
0,3,18
1,4,331
2,5,1558
3,6,328
4,7,65
5,8,3


In [31]:
df_outfit_products_sel['des_product_class'].value_counts() / df_outfit_products_sel['des_product_class'].count()

des_product_class
Footwear                               0.198278
Earrings                               0.198278
Bottoms                                0.171330
Tops                                   0.171330
Bags                                   0.150667
Outerwear                              0.053724
Dresses, jumpsuits and Complete set    0.026948
Ring                                   0.015411
Necklace                               0.007318
Glasses                                0.006715
Name: count, dtype: float64